# Implement webmaps with data downloads on District Digest
* Issue #1559
* Reference `ca_transit_speed_maps/05_webmap_example`
* Maps are here: https://console.cloud.google/storage/browser/calitp-map-tiles/caltrans_district_digest?pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))&csesidx=1815466960&project=cal-itp-data-infra

In [1]:
import _ct_district_grain_data_prep as _ct_district_data_prep
import branca
import branca.colormap as cm
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp_data_analysis import geography_utils
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from shared_utils import catalog_utils, webmap_utils

gcsgp = GCSGeoPandas()

In [2]:
district = "03 - Marysville / Sacramento"

In [3]:
district_int = [int(s) for s in district.split() if s.isdigit()][0]

In [4]:
# Load Datasets
operator_df = _ct_district_data_prep.data_wrangling_operator_profile(district)

In [5]:
operator_gdf = _ct_district_data_prep.data_wrangling_operator_map(
    list(operator_df.analysis_name.unique())
)

In [6]:
ct_district_gdf = _ct_district_data_prep.load_ct_district(district_int)

In [7]:
transit_route_shs_gdf, transit_route_shs_table = (
    _ct_district_data_prep.final_transit_route_shs_outputs(15, district_int)
)
district_gdf = _ct_district_data_prep.load_ct_district(district_int)
gtfs_table_df = _ct_district_data_prep.create_gtfs_stats(operator_df)

In [8]:
shn_gdf = _ct_district_data_prep.load_buffered_shn_map(district_int)

In [23]:
# Files for webmaps
boundary_file = f"district_{district_int}_boundary"
transit_routes_file = f"district_{district_int}_transit_routes"
shn_file = f"district_{district_int}_shn"
transit_shn_file = f"district_{district_int}_transit_routes_shn"

## Routes in District

In [17]:
color_map = cm.linear.Spectral_11
# color_map = color_map.to_step(25)  # adjust the number of steps as needed

In [18]:
color_map

In [28]:
color_map = branca.colormap.LinearColormap(
    colors=color_map.colors[7:], vmin=0, vmax=operator_gdf.Number.max()
)

ValueError: You must provide at least 2 colors.

In [21]:
color_map

In [24]:
district_map = webmap_utils.set_state_export(
    district_gdf,
    subfolder = "caltrans_district_digest/",
    filename=boundary_file,
    map_title="District Map",
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [25]:
transit_routes = webmap_utils.set_state_export(
    operator_gdf,
    subfolder = "caltrans_district_digest/",
    filename=transit_routes_file,
    map_title="Transit Routes",
    cmap=color_map,
    color_col="Number",
    existing_state=district_map
    
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [26]:
webmap_utils.render_spa_link(transit_routes["spa_link"], text="Open Routes for all Operators Map")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJEaXN0cmljdCBNYXAiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL2NhbHRyYW5zX2Rpc3RyaWN0X2RpZ2VzdC9kaXN0cmljdF8zX2JvdW5kYXJ5Lmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19LCB7Im5hbWUiOiAiVHJhbnNpdCBSb3V0ZXMiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL2NhbHRyYW5zX2Rpc3RyaWN0X2RpZ2VzdC9kaXN0cmljdF8zX3RyYW5zaXRfcm91dGVzLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19XSwgImxhdF9sb24iOiBbMzguODg3MDExNTA1MzcyMDgsIC0xMTkuNTA1NTAzNDkwMTc1NjFdLCAiem9vbSI6IDEzfQ==" target="_blank">Open Open Routes for all Operators Map in New Tab</a>

## Routes on SHN

In [47]:
color_map2 = cm.linear.RdYlBu_11.scale()
# adjust the number of steps as needed

In [48]:
color_map2 = branca.colormap.LinearColormap(
    colors=color_map2.colors[7:], vmin=0, vmax=100
)

In [49]:
color_map2

In [34]:
shn_map = webmap_utils.set_state_export(
    shn_gdf,
    subfolder="caltrans_district_digest/",
    filename=shn_file,
    map_title="State Highway Network Map",
    map_type="state_highway_network",
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [50]:
color_map2.caption = "Percentage of Transit Route on SHN Across All Districts"

In [52]:
webmap_utils.export_legend(
    cmap=color_map2, filename="transit_route_pct.svg", inner_labels=[]
)

legend written to calitp-map-tiles/transit_route_pct.svg, public_url https://storage.googleapis.com/calitp-map-tiles/transit_route_pct.svg


In [54]:
transit_shn_map = webmap_utils.set_state_export(
    transit_route_shs_gdf,
    subfolder="caltrans_district_digest/",
    filename=transit_shn_file,
    map_title="Transit Routes on the State Highway Network",
    cmap=color_map2,
    color_col="Percentage of Transit Route on SHN Across All Districts",
    existing_state=shn_map,
    legend_url="https://storage.googleapis.com/calitp-map-tiles/transit_route_pct.svg",
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [55]:
webmap_utils.render_spa_link(
    transit_shn_map["spa_link"], text="Open Routes on State Highway System Map"
)

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTdGF0ZSBIaWdod2F5IE5ldHdvcmsgTWFwIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy9jYWx0cmFuc19kaXN0cmljdF9kaWdlc3QvZGlzdHJpY3RfM19zaG4uZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fSwgInR5cGUiOiAic3RhdGVfaGlnaHdheV9uZXR3b3JrIn0sIHsibmFtZSI6ICJUcmFuc2l0IFJvdXRlcyBvbiB0aGUgU3RhdGUgSGlnaHdheSBOZXR3b3JrIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy9jYWx0cmFuc19kaXN0cmljdF9kaWdlc3QvZGlzdHJpY3RfM190cmFuc2l0X3JvdXRlc19zaG4uZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszOC44NzU5NTIzNTk2MDExODYsIC0xMjEuMjkzNDYwODY3ODA1NDFdLCAiem9vbSI6IDEzLCAibGVnZW5kX3VybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90cmFuc2l0X3JvdXRlX3BjdC5zdmcifQ==" target="_blank">Open Open Routes on State Highway System Map in New Tab</a>

In [56]:
webmap_utils.display_spa_map(transit_shn_map["spa_link"])